# Data Loading

In [88]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import random
from sklearn.impute import KNNImputer
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import scipy.stats as stats
import multiprocessing



In [89]:
#load data
X = np.load("/Users/jiaweizhang/research/data/X.npy")
Y = np.load("/Users/jiaweizhang/research/data/Y.npy")
Z = np.load("/Users/jiaweizhang/research/data/Z.npy")
M = np.load("/Users/jiaweizhang/research/data/M.npy")
S = np.load("/Users/jiaweizhang/research/data/S.npy")

N = len(X)
display(pd.DataFrame(X))
display(pd.DataFrame(Y))
display(pd.DataFrame(Z))
display(pd.DataFrame(M))
display(pd.DataFrame(S))


,0,1,2,3,4
0,0.064303,0.200401,-0.066047,-0.118805,1.0
1,0.011304,-1.753569,-1.647847,0.036556,0.0
2,-0.217136,-0.068792,1.071360,2.214527,1.0
3,1.404191,1.588740,-0.250640,0.000873,1.0
4,-1.104891,-1.323987,-4.722530,-2.826018,1.0
...,...,...,...,...,...
19995,0.007050,-0.520510,-0.155303,0.157510,1.0
19996,-2.102105,-0.642383,-0.281999,-0.100378,0.0
19997,0.180215,-1.635013,3.087097,2.295690,0.0
19998,0.424435,-0.896164,-2.484461,-0.503157,0.0


,0,1,2
0,28.911263,10.333793,9.325982
1,-19.238279,-9.106465,36.568410
2,93.103903,65.384071,263.794425
3,63.707859,55.847312,192.324767
4,-307.940454,-23.617158,271.089708
...,...,...,...
19995,2.198755,3.656618,-0.053475
19996,-14.933274,-10.864220,29.279905
19997,70.176443,130.703867,525.488709
19998,-24.622980,-9.148864,42.948562


,0
0,1.0
1,0.0
2,1.0
3,1.0
4,0.0
...,...
19995,0.0
19996,0.0
19997,0.0
19998,0.0


,0,1,2
0,0.0,0.0,0.0
1,0.0,0.0,1.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
19995,0.0,0.0,0.0
19996,0.0,0.0,1.0
19997,0.0,0.0,0.0
19998,0.0,0.0,1.0


,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
19995,199.0
19996,199.0
19997,199.0
19998,200.0


# One shot framework

### Randomly split one dataframe to two datasets
split_df takes a pandas DataFrame as input and randomly splits it into two separate DataFrames with a specified proportion of the data in each split. The function shuffles the indices randomly and splits the DataFrame using the shuffled indices. It returns the two separate DataFrames as output.

In [122]:
def split_df_shuffle(df):
    # Set the proportion of data to be split
    split_proportion = 0.5

    # Set a random seed for reproducibility
    random.seed(23)

    # Get the indices for the split
    indices = df.index.tolist()
    num_rows = len(df)
    split_index = int(num_rows * split_proportion)

    # Shuffle the indices randomly
    random.shuffle(indices)

    # Get the randomly selected rows for each split
    split1_indices = indices[:split_index]
    split2_indices = indices[split_index:]

    # Split the original DataFrame into two separate DataFrames
    df1 = df.loc[split1_indices]
    df2 = df.loc[split2_indices]
    
    return df1,df2

#split based on strata
def split_df(df,index_S):

    # Sort the groups by the number of rows in each group
    sorted_df = df.sort_values(by = index_S, ascending=True)
    
    # Split the sorted groups into two equal-sized sets of 100 strata each
    df_set1 = sorted_df.iloc[:int(N/2),0 : index_S]
    df_set2 = sorted_df.iloc[int(N/2):N, 0 : index_S]

    #set the index of the two sets from zero to 1
    df_set1.index = range(len(df_set1))
    df_set2.index = range(len(df_set2))
    
    # Return the two sets of strata
    return df_set1, df_set2


### T-test for T(Z,Y)
the Wilcoxon rank sum test
$T(\mathbf{Z}, \mathbf{Y})=\sum_{n=1}^{N}Z_{n}\cdot \text{rank}(Y_{n})=\sum_{n=1}^{N}\{Z_{n}\cdot \sum_{n^{\prime}=1}^{N} \mathbf{1}(Y_{n}\geq Y_{n^{\prime}})\}$.

In [91]:
def T(z,y):

    #the Wilcoxon rank sum test
    n = len(z)
    t = 0
    #O(N^2) version
    """
    for n in range(N):
        rank = sum(1 for n_prime in range(N) if Y[n] >= Y[n_prime])
        T += Z[n] * rank
    """

    #O(N*Log(N)) version
    my_list = []
    for i in range(n):
        my_list.append((z[i],y[i]))
    sorted_list = sorted(my_list, key=lambda x: x[1])

    #Calculate
    for i in range(n):
        t += sorted_list[i][0] * (i + 1)
    
    return t

def getT(G, df):
    
    # Get the imputed data Y and indicator Z
    df_imputed = G.transform(df)
    y = df_imputed[:, Z.shape[1] + X.shape[1]:df_imputed.shape[1]]
    z = df_imputed[:, 0]
    
    z_tiled = np.tile(z, 3)

    # Concatenate the tiled versions of Z together
    new_z = np.concatenate((z_tiled,))
    new_y = y.flatten()

    #the Wilcoxon rank sum test
    t = T(new_z,new_y)

    return t



#### t-test

In [ ]:
def ttest(G, df):
    
    # Get the imputed data Y and indicator Z
    df_imputed = G.transform(df)
    Y_pred = df_imputed[:, Z.shape[1] + X.shape[1]:df_imputed.shape[1]]
    Z_shuffled = df_imputed[:, 0]

    # Get the t-statistics for T(Z,Y)
    treatment = Y_pred[Z_shuffled == 1].flatten()
    control = Y_pred[Z_shuffled == 0].flatten()

    t, p = stats.ttest_ind(treatment, control, equal_var=True)

    return t,p

## One Short Framework 


In [112]:
def one_shot_test(Z, X, M, Y, S, G1, G2,  L=10000):
    """
    A one-shot framework for testing H_0.

    Args:
    Z: 2D array of observed treatment indicators
    X: 2D array of observed covariates
    M: 2D array of observed missing indicators
    Y: 2D array of observed values for K outcomes
    G1: a function that takes (Z, X, M, Y_k) as input and returns the imputed value for outcome k
    G2: a function that takes (Z, X, M, Y_k) as input and returns the imputed value for outcome k
    L: number of Monte Carlo simulations (default is 10000)

    Returns:
    p1: 1D array of exact p-values for testing Fisher's sharp null in part 1
    p2: 1D array of exact p-values for testing Fisher's sharp null in part 2
    """

    #print train start
    print("Training start")

    # create data a whole data frame
    Y_masked = np.ma.masked_array(Y, mask=M)
    Y_masked = Y_masked.filled(np.nan)
    df = pd.DataFrame(np.concatenate((Z, X, Y_masked,S), axis=1))
    
    # randomly split the data into two parts
    df1, df2 = split_df(df, X.shape[1] + Y.shape[1] + Z.shape[1])

    # impute the missing values and calculate the observed test statistics in part 1
    G1.fit(df1)
    t1_obs = getT(G1, df1)

    # impute the missing values and calculate the observed test statistics in part 2
    G2.fit(df2)
    t2_obs = getT(G2, df2)

    #print train end
    print("Training end")

    # simulate data and calculate test statistics
    t1_sim = np.zeros(L)
    t2_sim = np.zeros(L)
    
    for l in range(L):

        # simulate treatment indicators in parts 1 and 2
        df_sim = pd.DataFrame(np.concatenate((X, Y_masked, S), axis=1))
        
        # split the simulated data into two parts
        df1_sim, df2_sim = split_df(df_sim, X.shape[1] + Y.shape[1])

        # simulate treatment indicators in parts 1 and 2
        Z_1 = np.random.binomial(1, 0.5, df1_sim.shape[0]).reshape(-1, 1)
        Z_2 = np.random.binomial(1, 0.5, df2_sim.shape[0]).reshape(-1, 1)
        df1_sim = pd.concat([pd.DataFrame(Z_1), df1_sim], axis=1)
        df2_sim = pd.concat([pd.DataFrame(Z_2), df2_sim], axis=1)
        
    
        # get the test statistics in part 1
        t1_sim[l] = getT(G2, df1_sim)

        # get the test statistics in part 2
        t2_sim[l] = getT(G1, df2_sim)

        # Calculate the completeness percentage
        if l % 100 == 0:
            completeness = l / L * 100  
            print(f"Task is {completeness:.2f}% complete.")

    # calculate exact p-values for each outcome
    p1 = np.mean(t1_sim >= t1_obs, axis=0)
    p2 = np.mean(t2_sim >= t2_obs, axis=0)
    
    return p1, p2
    



###  Parallel Computing Version

In [ ]:
def worker(args):
    # unpack the arguments
    Z, X, M, S, Y_masked, G1, G2, t1_obs, t2_obs, shape, L = args

    # simulate data and calculate test statistics
    t1_sim = np.zeros(L)
    t2_sim = np.zeros(L)

    for l in range(L):

        # simulate treatment indicators in parts 1 and 2
        df_sim = pd.DataFrame(np.concatenate((X, Y_masked, S), axis=1))
        
        # split the simulated data into two parts
        df1_sim, df2_sim = split_df(df_sim, index_S = X.shape[1] + Y.shape[1])

        # simulate treatment indicators in parts 1 and 2
        Z_1 = np.random.binomial(1, 0.5, df1_sim.shape[0]).reshape(-1, 1)
        Z_2 = np.random.binomial(1, 0.5, df2_sim.shape[0]).reshape(-1, 1)
        df1_sim = pd.concat([pd.DataFrame(Z_1), df1_sim], axis=1)
        df2_sim = pd.concat([pd.DataFrame(Z_2), df2_sim], axis=1)

        # get the test statistics in part 1
        t1_sim[l] = getT(G2, df1_sim)

        # get the test statistics in part 2
        t2_sim[l] = getT(G1, df2_sim)

        # Calculate the completeness percentage
        if l % 100 == 0:
            completeness = l / L * 100  
            print(f"Task is {completeness:.2f}% complete.")

    p1 = np.mean(t1_sim >= t1_obs, axis=0)
    p2 = np.mean(t2_sim >= t2_obs, axis=0)

    return p1, p2

def one_shot_test_parallel(Z, X, M, Y, S, G1, G2, L=10000, n_jobs=multiprocessing.cpu_count()):
    """
    A one-shot framework for testing H_0.

    Args:
    Z: 2D array of observed treatment indicators
    X: 2D array of observed covariates
    M: 2D array of observed missing indicators
    Y: 2D array of observed values for K outcomes
    G1: a function that takes (Z, X, M, Y_k) as input and returns the imputed value for outcome k
    G2: a function that takes (Z, X, M, Y_k) as input and returns the imputed value for outcome k
    L: number of Monte Carlo simulations (default is 10000)

    Returns:
    p1: 1D array of exact p-values for testing Fisher's sharp null in part 1
    p2: 1D array of exact p-values for testing Fisher's sharp null in part 2
    """
    #print train start
    print("Training start")

    # create data a whole data frame
    Y_masked = np.ma.masked_array(Y, mask=M)
    Y_masked = Y_masked.filled(np.nan)
    df = pd.DataFrame(np.concatenate((Z, X, Y_masked, S), axis=1))
    
    # randomly split the data into two parts
    df1, df2 = split_df(df, index_S = Z.shape[1] + X.shape[1] + Y.shape[1])

    # impute the missing values and calculate the observed test statistics in part 1
    G1.fit(df1)
    t1_obs = getT(G1, df1)

    # impute the miassing values and calculate the observed test statistics in part 2
    G2.fit(df2)
    t2_obs = getT(G2, df2)

    #print train end
    print("Training end")
    
    # print the number of cores
    print(f"Number of cores: {n_jobs}")


    # simulate data and calculate test statistics in parallel
    args_list = [(Z, X, M, Y_masked, S, G1, G2, t1_obs, t2_obs, df.shape, int(L / n_jobs))] * n_jobs
    with multiprocessing.Pool(processes=n_jobs) as pool:
        p_list = pool.map(worker, args_list)
    p1 = np.mean([p[0] for p in p_list], axis=0)
    p2 = np.mean([p[1] for p in p_list], axis=0)
    
    return p1, p2


# Test the framework 

Test all the machine learning method in single core

In [ ]:
#MissForest
print("One-shot test for Fisher's sharp null")
missForest = IterativeImputer(estimator = RandomForestRegressor(),max_iter=10, random_state=0)

p1, p2 = one_shot_test(Z, X, M, Y, S,G1=missForest, G2=missForest)
print("p-values for part 1:", p1)
print("p-values for part 2:", p2)

In [113]:
#KNNimputer
print("One-shot test for Fisher's sharp null")
KNNimputer = KNNImputer(n_neighbors=7)
p1, p2 = one_shot_test(Z, X, M, Y,S, G1=KNNimputer, G2=KNNimputer)
print("p-values for part 1:", p1)
print("p-values for part 2:", p2)


One-shot test for Fisher's sharp null
Training start
Training end
              0         1         2         3    4          5          6  \
0      0.064303  0.200401 -0.066047 -0.118805  1.0  28.911263  10.333793   
71     1.890964  0.166024 -0.208788  0.361734  0.0   9.419433  25.350885   
70     1.370866 -0.746256 -0.561704  0.617559  1.0  40.690051  22.287257   
69    -1.335455 -0.778785 -2.252589 -0.231738  0.0 -21.747422 -10.736394   
68    -0.302942 -1.884908 -0.115570  0.898991  0.0  16.005350  -5.815427   
...         ...       ...       ...       ...  ...        ...        ...   
9930   1.204823 -0.789989 -0.063838 -0.017140  0.0   0.883652   3.294691   
9926   1.478240 -0.105537 -0.009922  0.641282  0.0   5.610774  18.818765   
9945   0.436497 -0.708884  0.015018  1.341464  1.0  45.574201  22.469670   
10065  0.652532  0.518161 -1.110980  0.059943  1.0   3.509592  10.112286   
10068  0.726724 -2.023163 -2.650801 -0.511687  1.0 -22.413458 -11.401134   

               7  
0 

In [ ]:
#BayesianRidge
print("One-shot test for Fisher's sharp null")
BayesianRidge = IterativeImputer(estimator = linear_model.BayesianRidge(),max_iter=10, random_state=0)
p1, p2 = one_shot_test(Z, X, M, Y, S, G1=BayesianRidge, G2=BayesianRidge)
print("p-values for part 1:", p1)
print("p-values for part 2:", p2)

In [ ]:
#Median imputer
print("One-shot test for Fisher's sharp null")
median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
p1, p2 = one_shot_test(Z, X, M, Y, S, G1=median_imputer, G2=median_imputer)
print("p-values for part 1:", p1)
print("p-values for part 2:", p2)

In [ ]:
#Mean imputer
print("One-shot test for Fisher's sharp null")
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
p1, p2 = one_shot_test(Z, X, M, Y, S, G1=mean_imputer, G2=mean_imputer)
print("p-values for part 1:", p1)
print("p-values for part 2:", p2)
